# W07 — Pandas: Czyszczenie danych

**Programowanie w Pythonie II** | Politechnika Opolska

Tematy:
1. Brakujące wartości — `isna()`, `dropna()`, `fillna()`
2. Duplikaty — `duplicated()`, `drop_duplicates()`
3. Konwersja typów — `astype()`, `pd.to_numeric()`, `pd.to_datetime()`
4. Operacje tekstowe — `str.lower()`, `str.strip()`, `str.replace()`, `str.contains()`

In [ ]:
import pandas as pd
import numpy as np

print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

---
## Część 1: Brakujące wartości

Dane działu HR — typowy eksport z systemu ERP z wieloma problemami.

In [ ]:
# Brudny dataset HR
data = {
    'id_pracownika': [1,2,3,4,5,6,7,8,9,10,
                      11,12,13,14,15,16,17,18,19,20,
                      3,7,12,18,5,
                      21,22,23,24,25],
    'imie': ['Anna', 'Bartek', 'CELINA', 'darek', 'Ewa',
             'Filip', 'Gosia', 'HENRYK', 'irena', 'Jan',
             'Kasia', 'Leszek', 'Marta', 'norbert', 'OLGA',
             'Piotr', 'Renata', 'sławek', 'Teresa', 'Urszula',
             'CELINA', 'Gosia', 'Leszek', 'sławek', 'Ewa',
             'Wanda', 'Xawery', 'Yvonne', 'Zbyszek', 'Agata'],
    'dzial': ['Sprzedaz', 'IT', 'HR', 'sprzedaz', 'IT',
              'HR', 'Sprzedaz', 'it', 'HR', 'Sprzedaz',
              'IT', 'HR', 'sprzedaz', 'IT', 'HR',
              'Sprzedaz', 'it', 'HR', 'Sprzedaz', 'IT',
              'HR', 'Sprzedaz', 'HR', 'HR', 'IT',
              'Sprzedaz', 'IT', 'hr', 'Sprzedaz', 'IT'],
    'wynagrodzenie': ['4500', '6200', '3800', '5100', '7500',
                      '4200', '5800', '6900', '3500', '4800',
                      '7200', '4100', '5500', '6800', '3900',
                      '5200', '4700', '6100', '3800', '5400',
                      '3800', '5800', '4100', '6100', '7500',
                      'brak', '5000', None, '4300', '6600'],
    'data_zatrudnienia': ['2020-03-15', '2019-07-22', '2021-01-10',
                          '2018-05-30', '2022-11-01', '2020-08-14',
                          '2019-12-05', '2021-06-18', '2017-09-23',
                          '2023-02-07', '2020-04-11', '2018-11-28',
                          '2022-07-15', '2019-03-19', '2021-10-08',
                          '2020-06-25', '2018-08-31', '2022-02-14',
                          '2019-05-07', '2021-09-20', '2021-01-10',
                          '2019-12-05', '2018-11-28', '2022-02-14',
                          '2022-11-01', '2023-01-15', '2022-05-20',
                          None, '2020-10-11', '2019-08-03'],
    'ocena_roczna': [4.5, 3.8, None, 4.2, 5.0,
                     3.5, 4.7, None, 4.1, 3.9,
                     5.0, 4.3, 3.6, None, 4.8,
                     3.7, 4.4, 4.9, None, 3.8,
                     None, 4.7, 4.3, 4.9, 5.0,
                     4.6, None, 4.9, 3.5, 4.1]
}

df = pd.DataFrame(data)
print(f"Shape: {df.shape}")
df.head(10)

In [ ]:
# Diagnoza — info() pokazuje typy i liczbę non-null
print("=== INFO ===")
df.info()

print("\n=== BRAKUJĄCE WARTOŚCI ===")
print(df.isna().sum())

print("\n=== PROCENT BRAKÓW ===")
print((df.isna().sum() / len(df) * 100).round(1))

In [ ]:
# isna() vs isnull() — dwa identyczne warianty nazwy
print("isna() == isnull():", df.isna().equals(df.isnull()))

# Wiersze z przynajmniej jednym NaN
wiersze_z_brakiem = df[df.isna().any(axis=1)]
print(f"\nWiersze z przynajmniej jednym NaN: {len(wiersze_z_brakiem)}")
wiersze_z_brakiem

In [ ]:
# dropna — usuwanie wierszy z NaN

# Domyślnie: usuwa wiersz jeśli MA JAKIKOLWIEK NaN
df_dropall = df.dropna()
print(f"Przed dropna: {len(df)}")
print(f"Po dropna (domyślnie): {len(df_dropall)}")

# subset= — usuwa tylko jeśli NaN w konkretnych kolumnach
df_drop_data = df.dropna(subset=['data_zatrudnienia'])
print(f"Po dropna(subset=['data_zatrudnienia']): {len(df_drop_data)}")

# thresh= — zostaw wiersze z co najmniej N wartościami non-null
df_thresh = df.dropna(thresh=5)
print(f"Po dropna(thresh=5): {len(df_thresh)}")

In [ ]:
# fillna — strategie uzupełniania NaN

# Strategia 1: stała wartość
df_s1 = df.copy()
df_s1['ocena_roczna'] = df_s1['ocena_roczna'].fillna(0)
print("Strategia 1 — fillna(0):")
print(df_s1['ocena_roczna'].describe())

# Strategia 2: średnia
df_s2 = df.copy()
srednia_ocena = df_s2['ocena_roczna'].mean()
df_s2['ocena_roczna'] = df_s2['ocena_roczna'].fillna(srednia_ocena)
print(f"\nStrategia 2 — fillna(mean={srednia_ocena:.2f}):")
print(f"NaN po fillna: {df_s2['ocena_roczna'].isna().sum()}")

# Strategia 3: mediana (lepsza przy wartościach odstających)
df_s3 = df.copy()
mediana_ocena = df_s3['ocena_roczna'].median()
df_s3['ocena_roczna'] = df_s3['ocena_roczna'].fillna(mediana_ocena)
print(f"\nStrategia 3 — fillna(median={mediana_ocena:.2f}):")
print(f"NaN po fillna: {df_s3['ocena_roczna'].isna().sum()}")

# Strategia 4: forward fill (przydatne w szeregach czasowych)
df_s4 = df.copy()
df_s4['ocena_roczna'] = df_s4['ocena_roczna'].ffill()
print(f"\nStrategia 4 — ffill():")
print(f"NaN po ffill: {df_s4['ocena_roczna'].isna().sum()}")

---
## Część 2: Duplikaty

In [ ]:
# duplicated() — wykrywanie duplikatów

print(f"Liczba zduplikowanych wierszy: {df.duplicated().sum()}")

# Pokaż duplikaty (wiersze które są kopiami wcześniejszych)
print("\nZduplikowane wiersze:")
print(df[df.duplicated()])

In [ ]:
# keep=False — oznacza WSZYSTKIE wystąpienia (oryginał + kopie)
print("Oryginały i duplikaty razem (keep=False):")
wszystkie_dup = df[df.duplicated(keep=False)].sort_values('id_pracownika')
print(wszystkie_dup)

# Duplikaty wg konkretnej kolumny
print(f"\nDuplikaty wg id_pracownika: {df.duplicated(subset=['id_pracownika']).sum()}")

# Ile razy pojawia się każde id
print("\nLiczność id_pracownika (tylko te > 1):")
print(df['id_pracownika'].value_counts()[df['id_pracownika'].value_counts() > 1])

In [ ]:
# drop_duplicates() — usuwanie duplikatów

przed = len(df)
df_bez_dup = df.drop_duplicates()
print(f"Przed: {przed}, po drop_duplicates(): {len(df_bez_dup)}")

# keep='last' — zostaw ostatnie wystąpienie
df_keep_last = df.drop_duplicates(keep='last')
print(f"Po drop_duplicates(keep='last'): {len(df_keep_last)}")

# Po usunięciu — reset indeksu
df_clean = df.drop_duplicates().reset_index(drop=True)
print(f"\nIndeks po reset_index: {df_clean.index.tolist()[:10]}...")
print(f"Shape: {df_clean.shape}")

---
## Część 3: Konwersja typów

In [ ]:
# Praca na df po usunięciu duplikatów
df_work = df.drop_duplicates().reset_index(drop=True).copy()

print("Typy danych:")
print(df_work.dtypes)
print()
print("Wynagrodzenie — unikalne wartości:")
print(sorted(df_work['wynagrodzenie'].unique(), key=lambda x: str(x)))

# Próba obliczenia średniej na stringu
try:
    print(df_work['wynagrodzenie'].mean())
except TypeError as e:
    print(f"Błąd mean() na stringu: {e}")

In [ ]:
# astype() wywali błąd gdy są 'brak' lub None
try:
    df_work['wynagrodzenie'].astype(float)
    print("astype działa!")
except ValueError as e:
    print(f"Błąd astype(float): {e}")

# Najpierw zamieniamy 'brak' na NaN
df_work['wynagrodzenie'] = df_work['wynagrodzenie'].replace('brak', np.nan)
print(f"\nPo replace('brak' → NaN): {df_work['wynagrodzenie'].isna().sum()} braków")

In [ ]:
# pd.to_numeric z errors='coerce' — bezpieczna konwersja
df_work['wynagrodzenie'] = pd.to_numeric(df_work['wynagrodzenie'], errors='coerce')

print(f"Typ po to_numeric: {df_work['wynagrodzenie'].dtype}")
print(f"NaN po konwersji: {df_work['wynagrodzenie'].isna().sum()}")

# Uzupełniamy braki medianą
mediana_wyn = df_work['wynagrodzenie'].median()
print(f"Mediana wynagrodzenia: {mediana_wyn}")

df_work['wynagrodzenie'] = df_work['wynagrodzenie'].fillna(mediana_wyn)
print(f"Po fillna(mediana): {df_work['wynagrodzenie'].isna().sum()} braków")
print(f"Średnie wynagrodzenie: {df_work['wynagrodzenie'].mean():.2f} PLN")

In [ ]:
# pd.to_datetime() — konwersja stringów na daty
print("Przed konwersją:")
print(df_work['data_zatrudnienia'].head(5).tolist())
print(f"Typ: {df_work['data_zatrudnienia'].dtype}")

df_work['data_zatrudnienia'] = pd.to_datetime(df_work['data_zatrudnienia'], errors='coerce')
print(f"\nTyp po konwersji: {df_work['data_zatrudnienia'].dtype}")
print(df_work['data_zatrudnienia'].head(5).tolist())

# .dt accessor — wydobycie elementów daty
df_work['rok_zatrudnienia'] = df_work['data_zatrudnienia'].dt.year
df_work['miesiac_zatrudnienia'] = df_work['data_zatrudnienia'].dt.month

print(f"\nLata zatrudnienia: {sorted(df_work['rok_zatrudnienia'].dropna().unique().astype(int).tolist())}")

In [ ]:
# astype('category') — dla danych kategorycznych
# Najpierw wyczyśćmy dzial (zapowiedź Materiału 4)
df_work['dzial'] = df_work['dzial'].str.strip().str.title()

print(f"Unikalne działy: {df_work['dzial'].unique()}")

# Konwersja na typ category
df_work['dzial_cat'] = df_work['dzial'].astype('category')
print(f"\nTyp po astype('category'): {df_work['dzial_cat'].dtype}")
print(f"Kategorie: {df_work['dzial_cat'].cat.categories.tolist()}")

# Porównanie pamięci
import sys
mem_str = df_work['dzial'].memory_usage(deep=True)
mem_cat = df_work['dzial_cat'].memory_usage(deep=True)
print(f"\nPamięć string: {mem_str} B")
print(f"Pamięć category: {mem_cat} B")
print(f"Oszczędność: {mem_str - mem_cat} B ({(1 - mem_cat/mem_str)*100:.0f}%)")

---
## Część 4: Operacje na tekstach (str accessor)

In [ ]:
# str. accessor — operacje tekstowe na całej kolumnie
print("Imiona przed czyszczeniem:")
print(df['imie'].tolist()[:15])

print("\nstr.lower():", df['imie'].str.lower().tolist()[:5])
print("str.upper():", df['imie'].str.upper().tolist()[:5])
print("str.title():", df['imie'].str.title().tolist()[:5])

# str.strip() — usuwa spacje z obu końców
test_spacje = pd.Series(['  Anna  ', 'Bartek ', ' CELINA', 'darek'])
print("\nstr.strip():")
print(test_spacje.str.strip().tolist())

In [ ]:
# str.replace() i str.contains()

# Normalizacja działów
df_str = df.copy()
print("Działy przed:")
print(df_str['dzial'].unique())

# Krok 1: strip + title
df_str['dzial'] = df_str['dzial'].str.strip().str.title()
print("\nPo title():", df_str['dzial'].unique())

# Krok 2: replace Title → wielkie litery (IT, HR)
df_str['dzial'] = df_str['dzial'].str.replace('Hr', 'HR', regex=False)
df_str['dzial'] = df_str['dzial'].str.replace('It', 'IT', regex=False)
print("Po replace:", df_str['dzial'].unique())

# str.contains() — filtrowanie po zawartości
it_depts = df_str[df_str['dzial'].str.contains('IT', na=False)]
print(f"\nPracownicy IT: {len(it_depts)}")

In [ ]:
# Kompletne czyszczenie tekstów
df_final = df.drop_duplicates().reset_index(drop=True).copy()

# Imiona: strip + title
df_final['imie'] = df_final['imie'].str.strip().str.title()
print("Imiona po czyszczeniu:")
print(df_final['imie'].tolist())

# Działy: strip + title + replace
df_final['dzial'] = df_final['dzial'].str.strip().str.title()
df_final['dzial'] = df_final['dzial'].str.replace('Hr', 'HR', regex=False)
df_final['dzial'] = df_final['dzial'].str.replace('It', 'IT', regex=False)
print(f"\nDziały po czyszczeniu: {sorted(df_final['dzial'].unique())}")
print(f"Liczba unikalnych działów: {df_final['dzial'].nunique()}")
print("\nLiczność per dział:")
print(df_final['dzial'].value_counts())

---
## Pełny pipeline czyszczenia

In [ ]:
# Kompletny pipeline: od brudnych danych do gotowych do analizy

df_pipeline = pd.DataFrame(data)  # świeża kopia
print(f"START: {df_pipeline.shape}")

# Krok 1: Napraw 'brak' → NaN, konwersja wynagrodzenia
df_pipeline['wynagrodzenie'] = df_pipeline['wynagrodzenie'].replace('brak', np.nan)
df_pipeline['wynagrodzenie'] = pd.to_numeric(df_pipeline['wynagrodzenie'], errors='coerce')
print(f"Krok 1 — NaN w wynagrodzenie: {df_pipeline['wynagrodzenie'].isna().sum()}")

# Krok 2: Usuń duplikaty
df_pipeline = df_pipeline.drop_duplicates().reset_index(drop=True)
print(f"Krok 2 — shape po dedup: {df_pipeline.shape}")

# Krok 3: Wypełnij NaN w wynagrodzeniu medianą
med = df_pipeline['wynagrodzenie'].median()
df_pipeline['wynagrodzenie'] = df_pipeline['wynagrodzenie'].fillna(med)
print(f"Krok 3 — użyta mediana: {med}, NaN: {df_pipeline['wynagrodzenie'].isna().sum()}")

# Krok 4: Wypełnij NaN w ocena_roczna średnią
avg = round(df_pipeline['ocena_roczna'].mean(), 2)
df_pipeline['ocena_roczna'] = df_pipeline['ocena_roczna'].fillna(avg)
print(f"Krok 4 — użyta średnia: {avg}, NaN: {df_pipeline['ocena_roczna'].isna().sum()}")

# Krok 5: Wyczyść imię
df_pipeline['imie'] = df_pipeline['imie'].str.strip().str.title()
print(f"Krok 5 — imiona: OK")

# Krok 6: Wyczyść dzial
df_pipeline['dzial'] = df_pipeline['dzial'].str.strip().str.title()
df_pipeline['dzial'] = df_pipeline['dzial'].str.replace('Hr', 'HR', regex=False)
df_pipeline['dzial'] = df_pipeline['dzial'].str.replace('It', 'IT', regex=False)
print(f"Krok 6 — działy: {sorted(df_pipeline['dzial'].unique())}")

# Krok 7: Konwersja daty
df_pipeline['data_zatrudnienia'] = pd.to_datetime(df_pipeline['data_zatrudnienia'], errors='coerce')
print(f"Krok 7 — dtype: {df_pipeline['data_zatrudnienia'].dtype}")

# Weryfikacja końcowa
print(f"\n=== WERYFIKACJA ===")
print(f"Shape: {df_pipeline.shape}")
print(f"NaN per kolumna:\n{df_pipeline.isna().sum()}")
print(f"\nDtypes:\n{df_pipeline.dtypes}")

In [ ]:
# Analiza biznesowa na czystych danych
print("=== RAPORT HR ===")
print("\nŚrednie wynagrodzenie per dział:")
print(df_pipeline.groupby('dzial')['wynagrodzenie'].mean().round(2))

print("\nŚrednia ocena per dział:")
print(df_pipeline.groupby('dzial')['ocena_roczna'].mean().round(2))

print("\nLiczba pracowników per dział:")
print(df_pipeline['dzial'].value_counts())

print("\nTop 3 najlepiej opłacanych:")
print(df_pipeline.nlargest(3, 'wynagrodzenie')[['imie', 'dzial', 'wynagrodzenie']])

---
## Podsumowanie

| Obszar | Metody | Kiedy używać |
|--------|--------|-------------|
| Brakujące wartości | `isna()`, `info()` | Diagnoza |
| Brakujące wartości | `dropna()` | Gdy wiersz bezużyteczny bez tej wartości |
| Brakujące wartości | `fillna(mediana)` | Liczby z wartościami odstającymi |
| Brakujące wartości | `fillna(srednia)` | Liczby z symetrycznym rozkładem |
| Brakujące wartości | `ffill()` | Szeregi czasowe |
| Duplikaty | `duplicated()` | Wykrycie |
| Duplikaty | `drop_duplicates()` | Usunięcie |
| Typy | `pd.to_numeric(errors='coerce')` | Liczby zapisane jako tekst |
| Typy | `pd.to_datetime(errors='coerce')` | Daty zapisane jako tekst |
| Typy | `astype('category')` | Kolumny z małą liczbą unikalnych wartości |
| Tekst | `str.strip()`, `str.title()` | Normalizacja formatowania |
| Tekst | `str.replace()` | Zamiana fragmentów tekstu |
| Tekst | `str.contains()` | Filtrowanie po zawartości |

**Na W08:** merge, groupby, pivot_table — analiza na czystych danych.